In [23]:
# !pip install --upgrade simpletransformers
# # !pip install --upgrade torch
# !pip install --upgrade transformers

In [24]:
!pip install cleantext

In [25]:
!pip install emoji

In [26]:
!pip install transformers

ERROR: Operation cancelled by user


In [27]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [28]:
import os
os.getcwd()
os.environ["WANDB_DISABLED"] = "true"

In [29]:
import random
import torch
import pandas as pd
import numpy as np
pd.options.display.max_colwidth = 200

def seed_all(seed_value):
    random.seed(seed_value) # Python
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False
seed_all(13)

In [30]:
import pandas as pd
import emoji
import warnings
import re
warnings.filterwarnings('ignore')
from cleantext import clean
from transformers import (AutoModel,AutoModelForMaskedLM, 
                          AutoTokenizer, LineByLineTextDataset,
                          DataCollatorForLanguageModeling,
                          Trainer, TrainingArguments)

In [31]:
def give_emoji_free_text(text):
    allchars = [str for str in text.encode().decode('utf-8')]
    emoji_list = [c for c in allchars if c in emoji.UNICODE_EMOJI]
    clean_text = ' '.join([str for str in text.encode().decode('utf-8').split() if not any(i in str for i in emoji_list)])
    return clean_text

def cleaning_text(text):
  text = text.lower()
  text = re.sub(r'[^\w\s]', ' ', text)
  text = re.sub(r'\d+', '', text)
  text = text.replace("&amp"," ")
  text = " ".join(text.split())
  return text


In [32]:
train_data=pd.read_csv("/content/gdrive/MyDrive/Hackathon/Train.csv")
train_data['tweet']=train_data['tweet'].apply(cleaning_text)
train_data['tweet']=train_data['tweet'].apply(give_emoji_free_text)


test_data=pd.read_csv("/content/gdrive/MyDrive/Hackathon/Test.csv")
test_data['tweet']=test_data['tweet'].apply(cleaning_text)
test_data['tweet']=test_data['tweet'].apply(give_emoji_free_text)


data = pd.concat([train_data,test_data])
data['tweet'] = data['tweet'].apply(lambda x: x.replace('\n',''))

text  = '\n'.join(data.tweet.tolist())

with open('text.txt','w') as f:
    f.write(text)

In [33]:
model_name = 'bert-base-uncased'
model = AutoModelForMaskedLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.save_pretrained('/content/gdrive/MyDrive/Hackathon/zindi_bert_base')

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.9.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/tra

('/content/gdrive/MyDrive/Hackathon/zindi_bert_base/tokenizer_config.json',
 '/content/gdrive/MyDrive/Hackathon/zindi_bert_base/special_tokens_map.json',
 '/content/gdrive/MyDrive/Hackathon/zindi_bert_base/vocab.txt',
 '/content/gdrive/MyDrive/Hackathon/zindi_bert_base/added_tokens.json',
 '/content/gdrive/MyDrive/Hackathon/zindi_bert_base/tokenizer.json')

In [34]:
data['length'] = data['tweet'].apply(lambda z:len(tokenizer.tokenize(z)))

In [35]:
data['length'].describe([0.7,0.8,0.9,0.95,0.99])

count    55231.000000
mean        40.902229
std         15.954082
min          3.000000
50%         45.000000
70%         53.000000
80%         56.000000
90%         59.000000
95%         61.000000
99%         66.000000
max        128.000000
Name: length, dtype: float64

In [36]:
train_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="text.txt", #mention train text file here
    block_size=80)

valid_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="text.txt", #mention valid text file here
    block_size=80)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.20)

training_args = TrainingArguments(
    output_dir="./zindi_bert_base_chk", #select model path for checkpoint
    overwrite_output_dir=True,
    num_train_epochs=15,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy= 'steps',
    save_total_limit=2,
    seed = 13,
    metric_for_best_model='eval_loss',
    greater_is_better=False,
    load_best_model_at_end =True,
    prediction_loss_only=True)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset)
trainer.train()
trainer.save_model(f'./zindi_bert_base')


Creating features from dataset file at text.txt
Creating features from dataset file at text.txt


KeyboardInterrupt: ignored